# Getting started with Classy Vision

Classy Vision is an end-to-end framework for image and video classification. Classy Vision makes it easy to write and launch distributed training jobs without having to worry about checkpointing, tensorboard logging and other features. Classy Vision already comes with a few computer vision baselines implemented, making it easy to make comparisions.

## What can Classy Vision do for me?

Here's a few examples of what Classy Vision handles for you in a typical training setting: 

 * Checkpointing
 * Tensorboard logging
 * Mixed-precision training
 * Configuration files
 * Distributed training
 * Metrics (e.g. top-5 accuracy)
 * Parameter scheduling
 * .... and much more!

## What about torchvision?

[`torchvision`](https://pytorch.org/docs/stable/torchvision/index.html) is a PyTorch package with common datasets, transforms and models used in computer vision research. While it provides many useful reference implementations, `torchvision` is designed to be a lightweight library and leaves a lot of up to the user. Classy Vision is an opinionated library built on top of `torchvision` that provide an end-to-end solution. After using Classy Vision for a while you will realize many Classy Vision abstractions simply use `torchvision` equivalents underneath, but play nicely with other classy components.

## Let's get started!

To start, let's create a project:

In [6]:
! classy-project my-project

INFO:root:Running classy project!
INFO:root:
    Successfully generated template project at '/Users/vini/fb/vreis/ClassyVision/tutorials/my-project'.
    To get started, run:
        $ cd my-project
        $ ./classy_train.py --config templates/config.json


In [9]:
import os
os.chdir('my-project')

In [12]:
!  ./classy_train.py --config configs/template_config.json --device cpu

INFO:root:Generic convolutional network trainer.
INFO:root:Done setting up distributed process_group with rank 0, world_size 1
INFO:root:Using CPU
            Could not compute FLOPs for model forward pass. Exception:
Traceback (most recent call last):
  File "/Users/vini/miniconda3/envs/cv_tutorials/lib/python3.6/site-packages/classy_vision/hooks/model_complexity_hook.py", line 39, in on_start
    input_shape=task.base_model.input_shape,
  File "/Users/vini/miniconda3/envs/cv_tutorials/lib/python3.6/site-packages/classy_vision/models/classy_model.py", line 260, in input_shape
    raise NotImplementedError
NotImplementedError
INFO:root:Number of parameters in model: 11689512
INFO:root:Advancing phase
INFO:root:Recreating data loader for new phase
INFO:root:Local unsynced metric values:
INFO:root:Rank: 0, train phase: 0, processed batches: 5
train loss: 4.874550373851912, LR rate: 0.1
Meters:
{'name': 'accuracy', 'value': {'top_1': 0.453125, 'top_5': 0.75}}
INFO:root:Local unsynced metr

That's it! You've launched your first training run. This trained a small MLP model on a dataset made of random noise, which is not that useful. The `classy-project` utility creates the scaffolding for you project, and you should modify it according to your needs. We'll learn how to customize your runs on the next few tutorials.

Let's take a look at what classy-project has created for us:

In [20]:
! find . | grep -v \.pyc | sort

.
./classy_train.py
./configs
./configs/template_config.json
./datasets
./datasets/__init__.py
./datasets/my_dataset.py
./losses
./losses/__init__.py
./losses/my_loss.py
./models
./models/__init__.py
./models/my_model.py


Here's what each folder means:

 * `configs`: stores your experiment configurations. Keeping all your experiments as separate configuration files helps making your research reproducible;
 * `models`: code for your custom model architectures;
 * `losses`: code for your custom loss functions;
 * `datasets`: code for your custom datasets;
 * `classy_train.py`: script to execute a training job; This uses the Classy Vision library to configure the job and execute it, and you might change it according to your needs;
 * `template_config.json`: experiment configuration file. This file is read by `classy_train.py` to configure your training job and launch it.

Let's take a peek at the configuration file:

In [13]:
! cat configs/template_config.json

{
    "name": "classification_task",
    "num_epochs": 2,
    "loss": {
        "name": "my_loss"
    },
    "dataset": {
        "train": {
            "name": "my_dataset",
            "split": "train",
            "crop_size": 224,
            "class_ratio": 0.5,
            "num_samples": 320,
            "seed": 0,
            "batchsize_per_replica": 32,
            "use_shuffle": true
        },
        "test": {
            "name": "my_dataset",
            "split": "val",
            "crop_size": 224,
            "class_ratio": 0.5,
            "num_samples": 100,
            "seed": 1,
            "batchsize_per_replica": 32,
            "use_shuffle": false
        }
    },
    "meters": {
        "accuracy": {
            "topk": [1, 5]
        }
    },
    "model": {
        "name": "my_model"
    },
    "optimizer": {
        "name": "sgd",
        "lr": {
            "name": "step",
            "values": [0.1, 0.01]
        },
        "weight_decay": 1e-4,
        "momen

That file can be shared with other researchers whenever you want them to reproduce your experiments. We generate `json` files by default but YAML is also supported. Take a look at the [Hydra tutorial](TODO) for an example of how to use YAML files within Classy Vision.

## Interactive development

Training scripts and configuration files are useful for running large training jobs on a GPU cluster (see our [AWS tutorial](TODO)), but a lot of day-to-day work happens interactively within Jupyter notebooks. Let's take a look at how to do the same training run as before, but within Jupyter.

In [24]:
import classy_vision

In [32]:
from classy_vision.dataset.transforms import ApplyTransformToKey
from torchvision.transforms import ToTensor
from datasets.my_dataset import MyDataset
from models.my_model import MyModel
from losses.my_loss import MyLoss

train_dataset = MyDataset(
    batchsize_per_replica=32,
    shuffle=False,
    transform=ApplyTransformToKey(ToTensor()),
    num_samples=100,
    crop_size=224,
    class_ratio=0.5,
    seed=0,
)

test_dataset = MyDataset(
    batchsize_per_replica=32,
    shuffle=False,
    transform=ApplyTransformToKey(ToTensor()),
    num_samples=100,
    crop_size=224,
    class_ratio=0.5,
    seed=0,
)


In [53]:
from classy_vision.tasks import ClassificationTask
from classy_vision.optim import SGD
from classy_vision.optim.param_scheduler import ConstantParamScheduler

model = MyModel()
loss = MyLoss()

# TODO: add defaults for momentum and weight decay
optimizer = SGD(
    lr_scheduler=ConstantParamScheduler(0.01), 
    momentum=0.99,
    weight_decay=0
)

from classy_vision.trainer import LocalTrainer

task = ClassificationTask() \
        .set_model(model) \
        .set_dataset(train_dataset, "train") \
        .set_dataset(test_dataset, "test") \
        .set_loss(loss) \
        .set_optimizer(optimizer) \
        .set_num_epochs(1)

trainer = LocalTrainer()
trainer.train(task)

That's it! Your model is trained now and ready for inference:

In [62]:
import torch
x = torch.randn((1, 3, 224, 224))
with torch.no_grad():
    y_hat = model(x)

# TODO: use a different model here so you actually get scores back
y_hat.shape

torch.Size([1, 1000])